In [1]:
import numpy as np
#import nrrd
from build import morse_complex as mc
from utils.functions import *
#from skel2graph import create_skeleton_graph, save_skeleton_graph
from util import patchify_voxel, unpatchify_voxel
#import pyvista
from skimage.measure import marching_cubes

# autoreload from .py files
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'build'

## Load Image

In [66]:
np.random.seed(0)
dist_img = np.random.rand(125,125,1)-0.5

In [3]:
img, header = nrrd.read('data/001.nrrd')

NameError: name 'nrrd' is not defined

In [ ]:
dist_img = distance_transform(1-img).astype(np.float32)

In [67]:
MC = mc.MorseComplex(dist_img[0:200,0:200,0:200])
MC.process_lower_stars(0, 2, 2, 2)
MC.get_number_of_critical_cells(0)

[[3080, 2066, 11, 0], [3080, 2066, 11, 0], [0, 0, 0, 0]]

In [68]:
MC.extract_morse_skeleton_below(threshold=0, dimension=3)

In [69]:
pixels_below = MC.get_morse_skeleton_below()
pixels_below = set(pixels_below)
len(pixels_below)

6108

In [70]:
MC.extract_morse_skeleton_parallel_below(threshold=0, dimension=3) #TODO: fix bug, hanging randomly

In [71]:
pixels_below = MC.get_morse_skeleton_below()
pixels_below = set(pixels_below)
len(pixels_below)

6108

In [72]:
MC.extract_morse_skeleton_batchwise_below(threshold=0, dimension=3, batches=200) #TODO: fix bug, hanging randomly

In [73]:
pixels_below = MC.get_morse_skeleton_below()
pixels_below = set(pixels_below)
len(pixels_below)

6108

In [77]:
MC = mc.MorseComplex(dist_img[0:200,0:200,0:200])
MC.process_lower_stars(0, 2, 2, 2)
MC.cancel_low_persistence_pairs_below(0,0.5)
MC.extract_morse_skeleton_batchwise_below(0,3,128)
pixels_below = MC.get_morse_skeleton_below()
pixels_below = set(pixels_below)
len(pixels_below)

1863

In [78]:
MC.get_number_of_critical_cells(0)

[[1079, 54, 0, 0], [1079, 54, 0, 0], [0, 0, 0, 0]]

In [79]:
MC = mc.MorseComplex(dist_img[0:200,0:200,0:200])
MC.process_lower_stars(0, 2, 2, 2)
MC.prepare_and_extract_morse_skeleton_below(0,0.5,[1,2,3],128)
MC.get_number_of_critical_cells(0)
pixels_below = MC.get_morse_skeleton_below()
pixels_below = set(pixels_below)
len(pixels_below)

1867

In [80]:
MC.get_number_of_critical_cells(0)

[[1079, 54, 0, 0], [1079, 54, 0, 0], [0, 0, 0, 0]]

## Params

In [ ]:
threshold = 0
epsilon = 0
delta = -1

## Do patch operation

In [ ]:
patch_size = (64, 64, 64)
pad = (1, 1, 1)
patch_list, start_ind, current_shape, orig_shape = patchify_voxel(dist_img, patch_size, pad)

## Run skeleton on patches

In [ ]:
skel_patch_list = []
print('number of patches:', len(patch_list))
for i, patch_ in enumerate(patch_list):
    # pad each side by 16 pixel with ones
    print('processing patch', i)
    d = patch_# distance_transform(1-patch_).astype(np.float32)
    
    MC = mc.MorseComplex(d)
    MC.process_lower_stars(2, 2, 2, 0)

    critical = MC.get_critical_cells()
    print(len(critical[0]), len(critical[1]), len(critical[2]), len(critical[3]))

    #################### Cancel pairs ####################
    # MC.cancel_pairs_below(threshold=threshold, print=True)

    #################### Extract Morse Skeleton ####################
    # MC.extract_morse_skeleton_below(threshold=threshold, dimension=3)
    # MC.extract_morse_skeleton_parallel_below(threshold=threshold, dimension=3) #TODO: fix bug, hanging randomly
    MC.extract_morse_skeleton_batchwise_below(threshold=threshold, dimension=3, batches=64) #TODO: fix bug, hanging randomly

    #################### Extract Morse Graph ####################
    pixels_below = np.array(MC.get_morse_skeleton_below())
    
    dmt_skeleton = np.zeros_like(d)
    if len(pixels_below.shape) == 2:
        dmt_skeleton[pixels_below[:,0], pixels_below[:,1], pixels_below[:,2]] = 1

    # crop the patch to the original size
    dmt_skeleton = dmt_skeleton[pad[0]:-pad[0], pad[1]:-pad[1], pad[2]:-pad[2]]
    skel_patch_list.append(dmt_skeleton)

## Reconstruct the whole skeleton from patch

In [ ]:
dmt_skeleton = unpatchify_voxel(skel_patch_list, start_ind, patch_size, current_shape, orig_shape)

## Save skeleton as graph

In [ ]:
dmt_skeleton_graph = create_skeleton_graph(dmt_skeleton)
save_skeleton_graph(dmt_skeleton_graph, 'images/DMTkeleton_skull.vtp')

## Save segmentation as mesh

In [ ]:
mesh_verts, faces, norms, vals = marching_cubes(img>0.0, level=0)
mesh_edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)

mesh_edges = np.concatenate((np.int32(2 * np.ones((mesh_edges.shape[0], 1))), mesh_edges), 1)
mesh = pyvista.UnstructuredGrid(mesh_edges.flatten(), np.array([4] * len(mesh_edges)), mesh_verts)
mesh_structured = mesh.extract_surface().clean()
mesh_structured.save('images/skull_seg.vtp')